# Output Parser

In [3]:
import openai
import os 
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,AIMessagePromptTemplate,HumanMessagePromptTemplate
from dotenv import load_dotenv

load_dotenv()


In [4]:
openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_base= os.getenv('OPENAI_API_BASE')
openai.api_type= "azure"
openai.api_version = os.getenv('OPENAI_API_VERSION')
deployment:str=os.getenv('CHATGPT_MODEL')

llm = AzureChatOpenAI(
    deployment_name=deployment,
    model_name=deployment
    )


In [5]:
from langchain.output_parsers import CommaSeparatedListOutputParser


In [6]:
csv_parser = CommaSeparatedListOutputParser()


In [7]:
csv_parser.get_format_instructions()


'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [8]:
human_template = "{instruction}\n{format_instructions}"
human_prompt = HumanMessagePromptTemplate.from_template(human_template)


In [9]:
chat = ChatPromptTemplate.from_messages([human_prompt])


In [10]:
chat.format_prompt(instruction="List the 5 citities with the highest populations", 
format_instructions=csv_parser.get_format_instructions()).to_messages()


[HumanMessage(content='List the 5 citities with the highest populations\nYour response should be a list of comma separated values, eg: `foo, bar, baz`')]

In [11]:
prompt = chat.format_prompt(instruction="List the 5 citities with the highest populations", 
format_instructions=csv_parser.get_format_instructions()).to_messages()


In [12]:
reply = llm(prompt)
reply.content


'Tokyo, Delhi, Shanghai, São Paulo, Mumbai'

In [13]:
csv_parser.parse(reply.content)


['Tokyo', 'Delhi', 'Shanghai', 'São Paulo', 'Mumbai']

In [14]:
list(reply.content)


['T',
 'o',
 'k',
 'y',
 'o',
 ',',
 ' ',
 'D',
 'e',
 'l',
 'h',
 'i',
 ',',
 ' ',
 'S',
 'h',
 'a',
 'n',
 'g',
 'h',
 'a',
 'i',
 ',',
 ' ',
 'S',
 'ã',
 'o',
 ' ',
 'P',
 'a',
 'u',
 'l',
 'o',
 ',',
 ' ',
 'M',
 'u',
 'm',
 'b',
 'a',
 'i']

In [15]:
from langchain.output_parsers import DatetimeOutputParser


In [16]:
datetime_parser = DatetimeOutputParser()


In [17]:
datetime_parser.get_format_instructions()


'Write a datetime string that matches the \n            following pattern: "%Y-%m-%dT%H:%M:%S.%fZ". Examples: 1019-04-01T15:48:21.024612Z, 0049-02-22T23:39:00.616111Z, 1610-03-02T15:57:51.948059Z'

In [18]:
human_text = "{instruction}\n{format_instructions}"

human_prompt = HumanMessagePromptTemplate.from_template(human_text)


In [19]:
system_prompt = SystemMessagePromptTemplate.from_template("You always give reponses with datetiem formats.")


In [20]:
chat = ChatPromptTemplate.from_messages([system_prompt, human_prompt])


In [21]:
prompt = chat.format_prompt(instruction="The date for when the Netscape browser was launched.", 
format_instructions=datetime_parser.get_format_instructions()).to_messages()


In [22]:
reply = llm(prompt)
reply.content


'The Netscape browser was launched on December 15, 1994.\nThe datetime string that matches the given pattern is "1994-12-15T00:00:00.000000Z".'

In [23]:
datetime_parser.parse(reply.content)


OutputParserException: Could not parse datetime string: The Netscape browser was launched on December 15, 1994.
The datetime string that matches the given pattern is "1994-12-15T00:00:00.000000Z".

In [ ]:
# pip install Pydantic


In [ ]:
from langchain.output_parsers import PydanticOutputParser


In [ ]:
from pydantic import BaseModel, Field


In [ ]:
class Playlist(BaseModel):
    name: str = Field(description="Name of the playlist")
    songs: list = Field(description="A list of songs in the playlist")


In [ ]:
parser = PydanticOutputParser(pydantic_object=Playlist)


In [ ]:
print(parser.get_format_instructions())


In [ ]:
human_text = "{instruction}\n{format_instructions}"


In [ ]:
human_prompt = HumanMessagePromptTemplate.from_template(human_text)


In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])


In [ ]:
prompt = chat_prompt.format_prompt(instruction="Create a playlist for disco songs",
format_instructions=parser.get_format_instructions()).to_messages()


In [ ]:
reply = llm(prompt)


In [ ]:
reply


In [ ]:
parser.parse(reply.content)
